In [1]:
#Import libraries, load watermark and set directories
from atgreen import *

In [ ]:
PATH=""

In [2]:
#Load dictionaries
#Features related
featuresToextract_greenblue=pickle.load(open(f"{PATH}/dicts/featuresToextract_greenblue.pickle", 'rb'))
featuresTodrop_greenblue=pickle.load(open(f"{PATH}/dicts/featuresTodrop_greenblue.pickle", 'rb'))
featuresToextract_confounders=pickle.load(open(f"{PATH}/dicts/featuresToextract_confounders.pickle", 'rb'))
featuresTodrop_confounders={}

#Osm-remapping related
osm_mask_values=pickle.load(open(f"{PATH}/dicts/osm_mask_values.pickle", 'rb'))
osm_mask_keys=pickle.load(open(f"{PATH}/dicts/osm_mask_keys.pickle", 'rb'))
osm_mask_elements=pickle.load(open(f"{PATH}/dicts/osm_mask_elements.pickle", 'rb'))
osm_mask_categories=pickle.load(open(f"{PATH}/dicts/osm_mask_categories.pickle", 'rb'))
osm_macroclasses=pickle.load(open(f"{PATH}/dicts/osm_macroclasses.pickle", 'rb'))

In [11]:
os.chdir(PATH)
db_params=pickle.load(open(f"{PATH}/dicts/db_params.pickle", 'rb'))
cities_list=query4table('cities_boundary','public', db_params)['city']


#Parameters
featuresToextract=featuresToextract_greenblue
featuresTodrop=featuresTodrop_greenblue
table='osm'

#Run through all cities
for ind, city in enumerate(cities_list):
    print(ind)
    print(city)

    filename=f"{PATH}/osm/cities/{city}.osm.pbf"

    gdfs_list=[]
    gdfs_list.append(waysExtraction(filename, featuresToextract, True, True))
    gdfs_list.append(relationsExtraction(filename, featuresToextract, True, True))
    gdf=pd.concat(gdfs_list)

    if len(featuresTodrop.keys())!=0:
        gdfs_list=[]
        gdfs_list.append(waysExtraction(filename, featuresTodrop, False, True))
        gdfs_list.append(relationsExtraction(filename, featuresTodrop, False, True))
        gdf_to_drop=pd.concat(gdfs_list)

    if len(gdf)!=0:
        if len(featuresTodrop.keys())!=0 and len(gdf_to_drop)!=0:
            gdf=gpd.overlay(gdf,gdf_to_drop, how='difference', keep_geom_type=True, make_valid=True)

        #Apply masks for categories
        gdf['category']=gdf['osm_value'].apply(lambda x: osm_mask_categories[osm_macroclasses[x]])
        gdf['osm_value']=gdf['osm_value'].apply(lambda x: osm_mask_values[x])
        gdf['osm_key']=gdf['osm_key'].apply(lambda x: osm_mask_keys[x])
        gdf['osm_element']=gdf['osm_element'].apply(lambda x: osm_mask_elements[x])
        gdf['city']=city

        boundary=query4cityboundary(city, db_params, 20000)

        gdf=gpd.overlay(gdf,boundary, how='intersection', keep_geom_type=True, make_valid=True)
        gdf.to_file(f"{path}/tmp/{city}.geojson", driver="GeoJSON")

        #Save into database
        geojson2db(f"osm.{city}",f"{path}/tmp/", city ,db_params, '')

        #Drop local file to save storage    
        os.remove(f"{path}/tmp/{city}.geojson")

        #Create index
        conn = psycopg2.connect(
               dbname=db_params['db_name'], user=db_params['db_user'], password=db_params['db_password'], host=db_params['db_host'])
        conn.autocommit=True
        sql=f"""CREATE INDEX idx_osm_{city} ON osm."{city}" USING gist (geom)"""
        conn.cursor().execute(sql)
        conn.close()
